In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import re
import PyQt4
import sys  
from PyQt5.QtGui import *  
from PyQt5.QtCore import *  
from PyQt5.QtWebKit import *  
from lxml import html
import datetime
import time
import selenium
from selenium import webdriver
from PyQt5.QtWebKitWidgets import QWebPage
from PyQt5.QtWidgets import QApplication

In [2]:
class Render(QWebPage):     
    def __init__(self, url):
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()   
    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit()

In [3]:
url = 'https://www.stitcher.com/stitcher-list/most-shared-podcasts'
r = Render(url)  
#result is a QString.
result = r.frame.toHtml()
#QString should be converted to string before processed by lxml
formatted_result = str(result.encode('ascii', 'ignore'))
#Next build lxml tree from formatted_result
tree = html.fromstring(formatted_result)
soup = BeautifulSoup(formatted_result,'lxml')

In [4]:
linklst = []
category_lst = []
week = []
for i in soup.find_all('span',class_ = 'sl-showName'):
    temp = str('https://www.stitcher.com') + str(i.a.get('href'))
    linklst.append(temp)
for q in soup.find_all('span',class_ = 'sl-category'):
    temp2 = str(q.text).replace('&amp;','&')
    category_lst.append(temp2)
wk = str(soup.find('div',id = 'date-text').option.text)
for i in range(0,100):
    week.append(wk)  

In [5]:
len(week)

100

In [6]:
index_list = []
for i in range(7,4000):
    if i % 4 == 0:
        index_list.append(i)

In [7]:
dict = {
    'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9, 
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
}

In [8]:
def monthToNum(shortMonth):
    return str(dict[shortMonth])

In [9]:
def get_newest_date(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content,'lxml')
    temp = str(soup.find('span',class_ = 'when').text)
    if len(temp.split(' ')) == 2:
        latest_date = ['2017']
        for i in temp.split(' '):
            if i in dict.keys():
                i = monthToNum(i)
                latest_date.append(i)
            else:
                i = i
                latest_date.append(i)
        if len(str(latest_date[1])) == 1:
            latest_date[1] = str(0) + str(latest_date[1])
        else:
            latest_date[1] = latest_date[1]
        if len(str(latest_date[2])) == 1:
            latest_date[2] = str(0) + str(latest_date[2])
        else:
            latest_date[2] = latest_date[2]
        res = latest_date[0]+'/'+ latest_date[1]+'/'+ latest_date[2]
        return [res]
    elif len(temp.split(' ')) == 3:
        days_difference = str(soup.find('span',{'class':'when'}).text.split(' ')[0])
        resu =  (datetime.datetime.now() - datetime.timedelta(days=int(days_difference))).date().strftime("%Y/%m/%d")
        return [resu]
    else:
        return [datetime.date.today().strftime("%Y/%m/%d")]

In [10]:
def get_total_episodes(link):
    episode_counter = 1
    r = requests.get(link)
    s = BeautifulSoup(r.content,'lxml')
    episode_counter += len(s.find_all('div',id = 'episodeContainer'))
    loopcount = 0
    if len(s.find_all('div',id = 'episodeContainer')) != 4:
        return [episode_counter]
    else:
        for n in index_list:
            lk = link +str('?episodes=1&newest=')+str(n)
            r = requests.get(lk)
            sp = BeautifulSoup(r.content,'lxml')
            length = len(sp.find_all('div',id = 'episodeContainer'))
            episode_counter += length
            #print loopcount
            loopcount += 1
            if length != 4:
                episode_counter += 4
                break
        return [int(episode_counter)]

In [11]:
def get_release_date(link):
    r = requests.get(link)
    s = BeautifulSoup(r.content,'lxml')
    loopcount = 0
    if len(s.find_all('div',id = 'episodeContainer')) == 0:
        return [str('Updates daily')]
    elif len(s.find_all('div',id = 'episodeContainer')) in [1,2,3]:
        release_date = str(s.find_all('span',class_ = 'when')[-1].text)
        mo = str(monthToNum(release_date.split(' ')[0]))
        if len(mo)==1:
            mo = str('0') + mo
        else:
            mo = mo
        day = release_date.split(' ')[1]
        if len(day) ==1:
            day = str('0') + day
        else:
            day = day
        res = '2017/' + str(mo) + '/' + str(day)
        return [res]
    else:
        release_date = str(s.find_all('span',class_ = 'when')[-1].text)
        for n in index_list:
            lk = link +str('?episodes=1&newest=')+str(n)
            r = requests.get(lk)
            sp = BeautifulSoup(r.content,'lxml')
            if len(sp.find_all('span',class_ = 'when')) in [1,2,3,4]:
                release_date = str(sp.find_all('span',class_ = 'when')[-1].text)
            elif len(sp.find_all('span',class_ = 'when')) == 0:
                break
        if len(str(release_date).split(' ')) == 2:
            mo = str(monthToNum(release_date.split(' ')[0]))
            if len(mo)==1:
                mo = str('0') + mo
            else:
                mo = mo
            day = release_date.split(' ')[1]
            if len(day) ==1:
                day = str('0') + day
            else:
                day = day
            res = '2017/' + str(mo) + '/' + str(day)
            return [res]
        elif len(str(release_date).split(' ')) == 3:
            if str(release_date).split(' ')[2] == 'ago':
                days_difference = str(release_date).split(' ')[0]
                resu =  (datetime.datetime.now() - datetime.timedelta(days=int(days_difference))).date().strftime("%Y/%m/%d")
                return [resu]
            else:
                mo = str(monthToNum(release_date.split(' ')[0]))
                if len(mo)==1:
                    mo = str('0') + mo
                else:
                    mo = mo
                day = str(release_date).split(' ')[1]
                if len(day) ==1:
                    day = str('0') + day
                else:
                    day = day
                year = str(release_date).split(' ')[2]
                res = str(year) + '/' + str(mo) + '/' + str(day)
                return [res]

In [12]:
def get_reviewcounts_and_stars(link):
    try:
        browser = webdriver.Chrome(r'D:\GA\chromedriver.exe')
        b = browser.get(link)
        soup = BeautifulSoup(browser.execute_script('return document.body.innerHTML'),'lxml')
        review_count = str(soup.find('span',itemprop = 'reviewCount').text)
        stars = str(soup.find('span',itemprop = 'ratingValue').text)
        return [review_count,stars]
    except:
        return ['N/A','N/A']

In [13]:
def get_name_and_description(link):
    r = requests.get(link)
    s = BeautifulSoup(r.content,'lxml')
    name = str(s.find('h1',class_ = 'showName').text).split('  ')[4]
#    description = s.find('p',class_ = 'about').contents[2].encode('utf8').split('  ')[5]
    description=s.find('p',class_='description').text
    return [name,description]

In [14]:
result = pd.DataFrame(columns=['Week','Name','Description','Category','review counts',
                               'stars','release date','total episodes','latest episode date','ranking','Link'])

In [15]:
os.chdir(r'D:\GA\Data')

In [16]:
%%time
loopcount = 0
for link in linklst[0:100]:
    try:
        n_and_d = get_name_and_description(link)
    except:
        pass
    print(str(loopcount) + str('.1'))
    try:
        rc_and_st = get_reviewcounts_and_stars(link)
    except:
        pass
    print(str(loopcount) + str('.2'))
    try:
        rd = get_release_date(link)
    except:
        pass
    print(str(loopcount) + str('.3'))
    try:
        te = get_total_episodes(link)
    except:
        pass
    print(str(loopcount) + str('.4'))
    try:
        le = get_newest_date(link)
    except:
        pass
    print(str(loopcount) + str('.5'))
    rk = loopcount + 1
    new_rk = [int(rk)]
    print(str(loopcount) + str('.6'))
    wk = [week[0]]
    print(str(loopcount) + str('.7'))
    cat = [category_lst[loopcount]]
    print(str(loopcount) + str('.8'))
    lk = [linklst[loopcount]]
    print(str(loopcount) + str('.9'))
    temp = wk + n_and_d + cat + rc_and_st + rd + te + le + new_rk + lk
    d = pd.DataFrame([temp],columns=['Week','Name','Description','Category','review counts',
                                   'stars','release date','total episodes','latest episode date','ranking','Link'])
    result = result.append(d)
    #filename = str('Stitcher Top Show ') + str(week[0]) + str('.csv')
    #result.to_csv(filename,encoding='utf8')     
    if loopcount%10 == 0:
        time.sleep(30)
    loopcount += 1

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.1
1.2
1.3
1.4
1.5
1.6
1.7
1.8
1.9
2.1
2.2
2.3
2.4
2.5
2.6
2.7
2.8
2.9
3.1
3.2
3.3
3.4
3.5
3.6
3.7
3.8
3.9
4.1
4.2
4.3
4.4
4.5
4.6
4.7
4.8
4.9
5.1
5.2
5.3
5.4
5.5
5.6
5.7
5.8
5.9
6.1
6.2
6.3
6.4
6.5
6.6
6.7
6.8
6.9
7.1
7.2
7.3
7.4
7.5
7.6
7.7
7.8
7.9
8.1
8.2
8.3
8.4
8.5
8.6
8.7
8.8
8.9
9.1
9.2
9.3
9.4
9.5
9.6
9.7
9.8
9.9
10.1
10.2
10.3
10.4
10.5
10.6
10.7
10.8
10.9
11.1
11.2
11.3
11.4
11.5
11.6
11.7
11.8
11.9
12.1
12.2
12.3
12.4
12.5
12.6
12.7
12.8
12.9
13.1
13.2
13.3
13.4
13.5
13.6
13.7
13.8
13.9
14.1
14.2
14.3
14.4
14.5
14.6
14.7
14.8
14.9
15.1
15.2
15.3
15.4
15.5
15.6
15.7
15.8
15.9
16.1
16.2
16.3
16.4
16.5
16.6
16.7
16.8
16.9
17.1
17.2
17.3
17.4
17.5
17.6
17.7
17.8
17.9
18.1
18.2
18.3
18.4
18.5
18.6
18.7
18.8
18.9
19.1
19.2
19.3
19.4
19.5
19.6
19.7
19.8
19.9
20.1
20.2
20.3
20.4
20.5
20.6
20.7
20.8
20.9
21.1
21.2
21.3
21.4
21.5
21.6
21.7
21.8
21.9
22.1
22.2
22.3
22.4
22.5
22.6
22.7
22.8
22.9
23.1
23.2
23.3
23.4
23.5
23.6
23.7
23.8
23.9
24.1
24.2


In [23]:
result

,Week,Name,Description,Category,review counts,stars,release date,total episodes,latest episode date,ranking,Link
0,02/25 - 03/03,The Joe Rogan Experience,"From comedian, UFC commentator and TV host, Jo...",Comedy,784,4.9,"2009/12/23,",1388,2019/03/08,1,https://www.stitcher.com/podcast/the-joe-rogan...
0,02/25 - 03/03,True Crime Garage,Each week Nic & The Captain get in the garage ...,Society & Culture,184,4.5,"2015/11/25,",286,2019/03/06,2,https://www.stitcher.com/podcast/true-crime-ga...
0,02/25 - 03/03,My Favorite Murder with Karen Kilgariff and Ge...,Lifelong fans of true crime stories Karen Kilg...,Comedy,170,4.1,"2016/01/13,",277,2019/03/07,3,https://www.stitcher.com/podcast/exactly-right...
0,02/25 - 03/03,Last Podcast On The Left,The Last Podcast On The Left covers all the ho...,"Pop Culture, TV & Film",79,4.1,"2015/02/16,",373,2019/03/06,4,https://www.stitcher.com/podcast/new-york-city...
0,02/25 - 03/03,The Daily,This is what the news should sound like. The b...,News & Politics,16,4.4,"2017/01/17,",548,2019/03/08,5,https://www.stitcher.com/podcast/the-new-york-...
0,02/25 - 03/03,The Bill Simmons Podcast,HBO and The Ringer's Bill Simmons hosts the mo...,Sports,12,4.9,"2015/09/25,",493,2019/03/08,6,https://www.stitcher.com/podcast/the-ringer/th...
0,02/25 - 03/03,The Rachel Maddow Show,Take MSNBC's popular Rachel Maddow Show with y...,News & Politics,627,4.9,"2018/10/29,",27,2019/03/08,7,https://www.stitcher.com/podcast/msnbc-rachel-...
0,02/25 - 03/03,This American Life,"From WBEZ Chicago Public Radio, This American ...",Society & Culture,296,4.9,"2018/11/11,",10,2019/03/03,8,https://www.stitcher.com/podcast/this-american...
0,02/25 - 03/03,Sword and Scale,The Sword and Scale true-crime podcast is an i...,Society & Culture,91,3.9,"2013/12/31,",137,2019/03/06,9,https://www.stitcher.com/podcast/wondery/sword...
0,02/25 - 03/03,Pod Save America,Four former aides to President Obama — Jon Fav...,News & Politics,34,4.4,"2017/01/6,",244,2019/03/08,10,https://www.stitcher.com/podcast/cadence13/pod...


In [24]:
link = linklst[0:100]

In [19]:
result['Link'] = linklst

In [20]:
result.to_csv('Stitcher Most Shared 0304-0310.csv')
#change and save every week